## Embeddings

In our previous example, we operated on high-dimensional bag-of-words vectors with the length of `vocab_size`, and we were explicitly converting from low-dimensional positional representation vectors into sparse one-hot representations. 

<img alt="diagram show how high-dimensional vectors are converted to embedding vectors" src="images/4-embedding-1.png" align="middle" />

The goal of using word embeddings and reducing the dimensionality are:
-  Finding the meaning of words based on their word approximation to other words.  This is done by taken two word vectors and analyzing how often the words in the vectors are used together.  The higher the frequency, the more you can find a correlation and relationship between the words.  
- This process of training the word embedding to find word approximations between words in a given dimension is how we reduce the word representation to low-dimensions.
-  Embedding vectors serve as numeric representations of words and are used as input to other machine learning network layers.
-  The embedding vector becomes the stored lookup table for words in the vocabulary

In this unit, we will continue exploring the **News AG** dataset. To begin, let's load the data and get some definitions from the previous unit.  In addition, we will allocation our training and testing datasets; word vocabulary size; and the category of our word classes: _World_, _Sports_, _Business_ and _Sci/Tech_

In [ ]:
!wget -q https://raw.githubusercontent.com/MicrosoftDocs/pytorchfundamentals/main/nlp-pytorch/torchnlp.py

In [2]:
import torch
import torchtext
from torchtext.data import get_tokenizer
import numpy as np
from torchnlp import *
from torchinfo import summary
train_dataset, test_dataset, classes, vocab = load_dataset()


Loading dataset...
Building vocab...


### Dealing with variable sequence size

When working with words, you are going to have text sequences or sentences that are of different lengths.  This can be problematic in training the word embeddings neural network. For consistency in the word embedding and improve training performance, we would have to apply some padding. This can be done using the `torch.nn.functional.pad` on a tokenized dataset. It adds zero values to the empty indices at the end of the vector.

<img alt="diagram show how showing padding" src="images/4-embedding-2.png" align="middle" />


In [3]:
def padify(b):
    # b is the list of tuples of length batch_size
    #   - first element of a tuple = label, 
    #   - second = feature (text sequence)
    # build vectorized sequence
    v = [encode(x[1]) for x in b]
    # first, compute max length of a sequence in this minibatch
    l = max(map(len,v))
    return ( # tuple of two tensors - labels and features
        torch.LongTensor([t[0]-1 for t in b]),
        torch.stack([torch.nn.functional.pad(torch.tensor(t),(0,l-len(t)),mode='constant',value=0) for t in v])
    )

Let's use the first 2 sentences as example to view the text length differences and effects of padding.

In [4]:
first_sentence = train_dataset[0][1]
second_sentence = train_dataset[1][1]

f_tokens = encode(first_sentence)
s_tokens = encode(second_sentence)

print(f'First Sentence in dataset:\n{first_sentence}')
print("Length:", len(train_dataset[0][1]))
print(f'\nSecond Sentence in dataset:\n{second_sentence}')
print("Length: ", len(train_dataset[1][1]))

First Sentence in dataset:
Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\band of ultra-cynics, are seeing green again.
Length: 144

Second Sentence in dataset:
Carlyle Looks Toward Commercial Aerospace (Reuters) Reuters - Private investment firm Carlyle Group,\which has a reputation for making well-timed and occasionally\controversial plays in the defense industry, has quietly placed\its bets on another part of the market.
Length:  266


Let's use the text sequence from the news article headlines in our dataset to change into a tokenize vector.  As you will see, the text sequence have different lengths.  We'll apply padding so all the text sequence will have a fixed length.  This approach is used when you have a large set of text sequences in your dataset.

- The length of the 1st and 2nd sentences displayed have difference lengths.  
- The max length of the dataset tensors is the length of the longest sentence length in the entire dataset.
- The zeros are added to the empty indexes in the tensor.

In [5]:
vocab_size = len(vocab)
labels, features = padify(train_dataset)  
print(f'features: {features}')

print(f'\nlength of first sentence: {len(f_tokens)}')
print(f'length of second sentence: {len(s_tokens)}')
print(f'size of features: {features.size()}')

features: tensor([[  432,   426,     2,  ...,     0,     0,     0],
        [15875,  1073,   855,  ...,     0,     0,     0],
        [   59,     9,   348,  ...,     0,     0,     0],
        ...,
        [ 7736,    63,   665,  ...,     0,     0,     0],
        [   97,    17,    10,  ...,     0,     0,     0],
        [ 2155,   223,  2405,  ...,     0,     0,     0]])

length of first sentence: 29
length of second sentence: 42
size of features: torch.Size([120000, 207])



### What is embedding?

The idea of **embedding** is the process of mapping words into vectors, which reflects the **_semantic meaning of a word_**. The length of its vectors are the embedding dimensions size. We will later discuss how to build meaningful word embeddings, but for now let's just think of embeddings as a way to lower dimensionality of a word vector. 

So, embedding layer would take a word as an input, and produce an output vector of specified `embedding_size`. In a sense, it is very similar to `Linear` layer, but instead of taking one-hot encoded vector, it will be able to take a word number as an input.

By using embedding layer as a first layer in our network, we can switch from bag-or-words to **embedding bag** model, where we first convert each word in our text into corresponding embedding, and then compute some aggregate function over all those embeddings, such as `sum`, `average` or `max`.  

<img alt="Image showing an embedding classifier for five sequence words." src="images/4-embedding-3.png" align="middle" />


Our classifier neural network will start with an embedding layer, then aggregation layer, and a linear classifier on top of it:
- `vocab_size` are the size of the total number of words we have in our vocabulary.
- `embed_dim` are the length of the word dimensions that show relationships between words passed as in the network.
- `num_class` are the number of news categories we are trying to classify (e.g. World, Sports, Business, Sci/Tech) 


In [6]:
class EmbedClassifier(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, num_class):
        super().__init__()
        self.embedding = torch.nn.Embedding(vocab_size, embed_dim)
        self.fc = torch.nn.Linear(embed_dim, num_class)

    def forward(self, x):
        x = self.embedding(x)
        x = torch.mean(x,dim=1)
        return self.fc(x)

### Training embedding classifier

Now we’ll define our training dataloader and use the `collate_fn` to apply the padify function to the datasets as they loaded in each batch.  As a result, the training dataset will be padded.

In [7]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=padify, shuffle=True)

We can train the model using the training function defined in the previous unit to run the embedding network.  The training output serves as a vector lookup store based on the unique index tokens from the vocabulary.

In [8]:
net = EmbedClassifier(vocab_size,32,len(classes)).to(device)
train_epoch(net,train_loader, lr=1, epoch_size=25000)

3200: acc=0.64875
6400: acc=0.69234375
9600: acc=0.7110416666666667
12800: acc=0.72421875
16000: acc=0.73625
19200: acc=0.7476041666666666
22400: acc=0.7541964285714285


(0.9072840441058861, 0.7583173384516955)

> **Note**: We are only training for 25k records here (less than one full epoch) for the sake of time, but you can continue training, write a function to train for several epochs, and experiment with learning rate parameter to achieve higher accuracy. You should be able to go to the accuracy of about 90%.

### EmbeddingBag Layer and Variable-Length Sequence Representation

In the previous architecture, we needed to pad all sequences to the same length in order to fit them into a minibatch. This is not the most efficient way to represent variable length sequences - another apporach would be to use **offset** vector, which would hold offsets of all sequences stored in one large vector.

<img alt="Image showing an offset sequence representation" src="images/4-embedding-4.png" align="middle" />

> **Note**: On the picture above, we show a sequence of characters, but in our example we are working with sequences of words. However, the general principle of representing sequences with offset vector remains the same.

To work with offset representation, we use PyTorch's `EmbeddingBag` layer. It is similar to `Embedding`, but it takes content vector and offset vector as input, and it also includes averaging layer, which can be `mean`, `sum` or `max`.

Here is modified network that uses `EmbeddingBag`:


In [9]:
class EmbedClassifier(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, num_class):
        super().__init__()
        self.embedding = torch.nn.EmbeddingBag(vocab_size, embed_dim)
        self.fc = torch.nn.Linear(embed_dim, num_class)

    def forward(self, text, off):
        x = self.embedding(text, off)
        return self.fc(x)

To prepare the dataset for training, we need to provide a conversion function that will prepare the offset vector:

In [10]:
def offsetify(b):
    # first, compute data tensor from all sequences
    x = [torch.tensor(encode(t[1])) for t in b]
    # now, compute the offsets by accumulating the tensor of sequence lengths
    o = [0] + [len(t) for t in x]
    o = torch.tensor(o[:-1]).cumsum(dim=0)
    return ( 
        torch.LongTensor([t[0]-1 for t in b]), # labels
        torch.cat(x), # text 
        o
    )

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=offsetify, shuffle=True)

The offset vector is calculated by first combining the sentences indices into one tensor sequence, then extracting the starting index location of each sentence in the sequence. For example: 
- The length of the first sentence in our training dataset is 29.  Meaning the first index of the offset will be `0`.
- The length of the second sentence in the dataset is 42.  Meaning the second index of the offset will be `29`, where the first sentence ended. 
- The third index of the offset will be 29 + 42 = `71`, where the 2nd sentence ended.

In [11]:
labels, features, offset = offsetify(train_dataset)  
print(f'offset: {offset}')
print(f'\nlength of first sentence: {len(f_tokens)}')
print(f'length of second sentence: {len(s_tokens)}')
print(f'size of data vector: {features.size()}')
print(f'size of offset vector: {offset.size()}')

offset: tensor([      0,      29,      71,  ..., 5193441, 5193488, 5193569])

length of first sentence: 29
length of second sentence: 42
size of data vector: torch.Size([5193609])
size of offset vector: torch.Size([120000])


> **Note:** that unlike in all previous examples, our network now accepts two parameters: data vector and offset vector, which are of different sizes. Similarly, our data loader also provides us with 3 values instead of 2: both text and offset vectors are provided as features. Therefore, we need to slightly adjust our training function to take care of that:

In [12]:
net = EmbedClassifier(vocab_size,32,len(classes)).to(device)

def train_epoch_emb(net,dataloader,lr=0.01,optimizer=None,loss_fn = torch.nn.CrossEntropyLoss(),epoch_size=None, report_freq=200):
    optimizer = optimizer or torch.optim.Adam(net.parameters(),lr=lr)
    loss_fn = loss_fn.to(device)
    net.train()
    total_loss,acc,count,i = 0,0,0,0
    for labels,text,off in dataloader:
        optimizer.zero_grad()
        labels,text,off = labels.to(device), text.to(device), off.to(device)
        out = net(text, off)
        loss = loss_fn(out,labels) #cross_entropy(out,labels)
        loss.backward()
        optimizer.step()
        total_loss+=loss
        _,predicted = torch.max(out,1)
        acc+=(predicted==labels).sum()
        count+=len(labels)
        i+=1
        if i%report_freq==0:
            print(f"{count}: acc={acc.item()/count}")
        if epoch_size and count>epoch_size:
            break
    return total_loss.item()/count, acc.item()/count


train_epoch_emb(net,train_loader, lr=4, epoch_size=25000)

3200: acc=0.6496875
6400: acc=0.6853125
9600: acc=0.7097916666666667
12800: acc=0.725546875
16000: acc=0.737375
19200: acc=0.7446354166666667
22400: acc=0.7532589285714286


(22.52296015275112, 0.7591570697376839)


## Semantic Embeddings: Word2Vec

In our previous example, the model embedding layer learnt to map words to vector representation, however, this representation did not have much semantical meaning. It would be nice to learn such vector representation, that similar words or symonims would correspond to vectors that are close to each other in terms of some vector distance (eg. euclidian distance).

To do that, we need to pre-train our embedding model on a large collection of text in a specific way. One of the first ways to train semantic embeddings is called **Word2Vec**.   It helps **_map the probability of a word_**, based on the contexts from texts in the sequence.  It is based on two main architectures that are used to produce a distributed representation of words:

 - **Continuous bag-of-words** (CBoW) — in this architecture, we train the model to predict a word from surrounding context. Given the ngram $(W_{-2},W_{-1},W_0,W_1,W_2)$, the goal of the model is to predict $W_0$ from $(W_{-2},W_{-1},W_1,W_2)$.  For example:  **_"I like my hot dog on a __"_**.  Here the predicted word would be **_"bun"_**.
 - **Continuous skip-gram** is opposite to CBoW. The model uses surrounding window of context words to predict the current word.  For example: you can predict **_dog_** to be more associated with the word **_veterinary_**.

CBoW is faster, while skip-gram is slower, but does a better job of representing infrequent words.

<img alt="Image showing both CBoW and Skip-Gram algorithms to convert words to vectors." src="images/4-embedding-5.png" align="middle" />

Both CBOW and Skip-Grams are “predictive” embeddings, in that they only take local contexts into account. Word2Vec does not take advantage of global context.   

- **GloVe** (Global Vectors) - derives the relationship between words. It determines how often a particular word-pair occurs together in a text by leveraging the idea of co-occurence matrix and uses neural methods to decompose co-occurrence matrix into more expressive and non linear word vectors.
- **FastText** - builds on Word2Vec by learning vector representations for each word and the charachter n-grams found within each word. The values of the representations are then averaged into one vector at each training step. While this adds a lot of additional computation to pre-training, it enables word embeddings to encode sub-word information.
- **Gensim** (Generate Similar) - is an open source NLP Python library that provides a unified interface to build word vectors, corpus, perform topic identification, and other NLP tasks.

FastText and GloVe are other word embeddings techniques that predict the probably of words appearing together.  

In our Word2Vec examples, we'll using pre-trained semantic embeddings, but it is interesting to see how those embeddings can be trained using either FastText, CBoW, or Skip-gram architectures. This exercise goes beyond this module, but those interested can reference Word Embeddings tutorials on Pytorch's website. 

### Genim

The **gensim** framework can be used with Pytorch to train most commonly used embeddings in a few lines of code.  To experiment with word2vec embedding pre-trained on Google News dataset, we can use the **gensim** library. Below we find the words that are most similar to 'neural'

> **Note:** When you first create word vectors, downloading them can take some time!

In [13]:
import gensim.downloader as api
w2v = api.load('word2vec-google-news-300')

Let's take a look at words that are similar to 'dog'.

In [14]:
for w,p in w2v.most_similar('dog'):
    print(f"{w} -> {p}")

dogs -> 0.8680489659309387
puppy -> 0.8106428384780884
pit_bull -> 0.780396044254303
pooch -> 0.7627377510070801
cat -> 0.7609456777572632
golden_retriever -> 0.7500902414321899
German_shepherd -> 0.7465174198150635
Rottweiler -> 0.7437614798545837
beagle -> 0.7418621778488159
pup -> 0.740691065788269


We can also extract vector embeddings from the word, to be used in training classification model (we'll only show first 20 components of the vector for clarity):

In [15]:
w2v.word_vec('play')[:20]

array([ 0.01226807,  0.06225586,  0.10693359,  0.05810547,  0.23828125,
        0.03686523,  0.05151367, -0.20703125,  0.01989746,  0.10058594,
       -0.03759766, -0.1015625 , -0.15820312, -0.08105469, -0.0390625 ,
       -0.05053711,  0.16015625,  0.2578125 ,  0.10058594, -0.25976562],
      dtype=float32)

The great thing about semantical embeddings is that you can manipulate vector encoding to change the semantics. For example, we can ask to find a word, whose vector representation would be as close as possible to words *king* and *woman*, and as far away from the word *man*:

In [16]:
w2v.most_similar(positive=['king','woman'],negative=['man'])[0]

('queen', 0.7118192911148071)

## Using Pre-Trained Embeddings in PyTorch

We can modify the example above to pre-populate the matrix in our embedding layer with semantical embeddings, such as Word2Vec. We need to take into account that vocabularies of pre-trained embedding are an addition to the existing text corpus that we already have so they will likely not match. As a result, we will initialize weights for the missing words with random values:

In [17]:
embed_size = len(w2v.get_vector('hello'))
print(f'Embedding size: {embed_size}')

net = EmbedClassifier(vocab_size,embed_size,len(classes))

print('Populating matrix, this will take some time...',end='')
found, not_found = 0,0
for i,w in enumerate(vocab.itos):
    try:
        net.embedding.weight[i].data = torch.tensor(w2v.get_vector(w))
        found+=1
    except:
        net.embedding.weight[i].data = torch.normal(0.0,1.0,(embed_size,))
        not_found+=1

print(f"Done, found {found} words, {not_found} words missing")
net = net.to(device)

Embedding size: 300
Populating matrix, this will take some time...Done, found 41080 words, 54732 words missing


Now let's train our model. Note that the time it takes to train the model is significantly larger than in the previous example, due to larger embedding layer size, and thus much higher number of parameters. Also, because of this, we may need to train our model on more examples if we want to avoid overfitting.

In [18]:
train_epoch_emb(net,train_loader, lr=4, epoch_size=25000)

3200: acc=0.6409375
6400: acc=0.6875
9600: acc=0.7163541666666666
12800: acc=0.730859375
16000: acc=0.740875
19200: acc=0.7510416666666667
22400: acc=0.7584821428571429


(228.52019353806782, 0.7641154830454254)

In our case we do not see a huge increase in accuracy, which is likely due to the quite different vocalularies. 
To overcome the problem of different vocabularies, we can use one of the following solutions:
* Re-train word2vec model on our vocabulary
* Load our dataset with the vocabulary from the pre-trained word2vec model. Vocabulary used to load the dataset can be specified during loading.

The latter approach seems easier, especially because PyTorch `torchtext` framework contains built-in support for embeddings. 

### GloVe Embeddings

To load our dataset with the vocabulary from a pre-trained word2vec model, we use Glove embeddings. We'll start by instantiating GloVe-based vocabulary in the following manner:

In [ ]:
vocab = torchtext.vocab.GloVe(name='6B', dim=50)

Loaded vocabulary has the following basic operations:
* `vocab.stoi` dictionary allows us to convert word into its dictionary index
* `vocab.itos` does the opposite - converts number into word
* `vocab.vectors` is the array of embedding vectors, so to get the embedding of a word we need to use `vocab.vectors[vocab.stoi[s]]`

Here is the example of manipulating embeddings to demonstrate the equation **kind-man+woman = queen** (the coefficient was tweak a bit to make it work):

In [20]:
# get the vector corresponding to kind-man+woman
qvec = vocab.vectors[vocab.stoi['king']]-vocab.vectors[vocab.stoi['man']]+1.3*vocab.vectors[vocab.stoi['woman']]
# find the index of the closest embedding vector 
d = torch.sum((vocab.vectors-qvec)**2,dim=1)
min_idx = torch.argmin(d)
# find the corresponding word
vocab.itos[min_idx]

'queen'

To train the classifier using those embeddings, we first need to encode our dataset using GloVe vocabulary:

In [21]:
def offsetify(b):
    # first, compute data tensor from all sequences
    x = [torch.tensor(encode(t[1],voc=vocab)) for t in b] # pass the instance of vocab to encode function!
    # now, compute the offsets by accumulating the tensor of sequence lengths
    o = [0] + [len(t) for t in x]
    o = torch.tensor(o[:-1]).cumsum(dim=0)
    return ( 
        torch.LongTensor([t[0]-1 for t in b]), # labels
        torch.cat(x), # text 
        o
    )

As we have seen above, all vector embeddings are stored in `vocab.vectors` matrix. It makes it super-easy to load those weights into weights of embedding layer using simple copying:

In [22]:
net = EmbedClassifier(len(vocab),len(vocab.vectors[0]),len(classes))
net.embedding.weight.data = vocab.vectors
net = net.to(device)

Now let's train our model and see if we get better results:

In [23]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=offsetify, shuffle=True)
train_epoch_emb(net,train_loader, lr=4, epoch_size=25000)

3200: acc=0.6359375
6400: acc=0.68203125
9600: acc=0.706875
12800: acc=0.727734375
16000: acc=0.738625
19200: acc=0.7465104166666666
22400: acc=0.7526785714285714


(35.71297184900832, 0.7573576455534229)

One of the reasons we are not seeing a significant increase in accuracy is due to the fact that some words from our dataset are missing in the pre-trained GloVe vocabulary, and thus they are essentially ignored. To overcome this fact, we can train our own embeddings on our dataset.

## Contextual Embeddings

One key limitation of traditional pretrained embedding representations such as Word2Vec is the problem of word sense and removing ambiguity by making them clear. While pretrained embeddings can capture some of the meaning of words in context, every possible meaning of a word is encoded into the same embedding. This can cause problems in downstream models, since many words such as the word 'play' have different meanings depending on the context they are used in.

For example, the word 'play' in these two different sentences have quite different meaning:
- I went to a **play** at the theatre.
- John wants to **play** with his friends.

The pretrained embeddings above represent both meanings of the word 'play' in the same embedding. To overcome this limitation, we need to build embeddings based on the **language model**, which is trained on a large corpus of text, and *knows* how words can be put together in different contexts. Discussing contextual embeddings is out of scope for this tutorial, but we will come back to them when talking about language models in the next unit.
